# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../output_data/CityWeatherData.csv", encoding="utf-8")
cities.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed,Cloudiness (%),Humidity (%),Latitude,Longitude,Max Temp (F)
0,barrow,90,US,1970-01-01 00:00:01.603248776,86,71.29,-156.79,28.40,13.87,90.0,86.0,71.29,-156.79,28.40
1,kapaa,90,US,1970-01-01 00:00:01.603248637,83,22.08,-159.32,82.00,9.17,90.0,83.0,22.08,-159.32,81.00
2,sao joao da barra,74,BR,1970-01-01 00:00:01.603248762,86,-21.64,-41.05,72.46,9.15,66.0,87.0,-21.64,-41.05,72.23
3,dois vizinhos,0,BR,1970-01-01 00:00:01.603248800,78,-25.73,-53.06,66.51,2.51,0.0,77.0,-25.73,-53.06,66.00
4,talnakh,63,RU,1970-01-01 00:00:01.603248801,84,69.49,88.40,8.60,2.24,49.0,92.0,69.49,88.40,6.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
humidity = cities['Humidity (%)'].astype(float)
max_humidity = cities['Humidity (%)'].max()
max_humidity = max_humidity.astype(float)
locations = cities[['Lat', 'Lng']]
max_humidity.dtype

dtype('float64')

In [16]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness (%)"] == 0) & (cities['Windspeed'] < 10), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed,Cloudiness (%),Humidity (%),Latitude,Longitude,Max Temp (F)
0,bandarbeyla,0,SO,1970-01-01 00:00:01.603248823,68,9.49,50.81,78.37,8.90,0.0,68.0,9.49,50.81,78.67
1,swan hill,0,AU,1970-01-01 00:00:01.603248842,33,-35.34,143.55,75.29,7.14,0.0,30.0,-35.34,143.55,76.75
2,adrar,0,DZ,1970-01-01 00:00:01.603248853,29,25.75,-1.00,71.47,9.64,0.0,30.0,25.75,-1.00,70.54
3,quelimane,0,MZ,1970-01-01 00:00:01.603248862,73,-17.88,36.89,74.28,5.82,0.0,88.0,-17.88,36.89,77.00
4,khandbari,0,NP,1970-01-01 00:00:01.603248896,53,27.37,87.20,72.50,1.68,0.0,49.0,27.37,87.20,75.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed,Cloudiness (%),Humidity (%),Latitude,Longitude,Max Temp (F),Hotel Name
0,bandarbeyla,0,SO,1970-01-01 00:00:01.603248823,68,9.49,50.81,78.37,8.90,0.0,68.0,9.49,50.81,78.67,Bandarbeyla
1,swan hill,0,AU,1970-01-01 00:00:01.603248842,33,-35.34,143.55,75.29,7.14,0.0,30.0,-35.34,143.55,76.75,Swan Hill
2,adrar,0,DZ,1970-01-01 00:00:01.603248853,29,25.75,-1.00,71.47,9.64,0.0,30.0,25.75,-1.00,70.54,Reggane
3,quelimane,0,MZ,1970-01-01 00:00:01.603248862,73,-17.88,36.89,74.28,5.82,0.0,88.0,-17.88,36.89,77.00,Quelimane
4,khandbari,0,NP,1970-01-01 00:00:01.603248896,53,27.37,87.20,72.50,1.68,0.0,49.0,27.37,87.20,75.56,Khandbari


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_city_df.iterrows()]
locations = narrow_city_df[["Lat", "Lng"]]

NameError: name 'narrow_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
